## Hopfield Modell

zunächst werdenen die benötigten Bibliotheken implementiert

In [9]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('MacOSX')

Jetzt wird zunächst das Hopfieldmodel implementiert

In [10]:
class HopfieldNetwork:
    #Initierungsfunktion
    def __init__(self, num_neurons):
        self.num_neurons = num_neurons
        self.matrix_size = int(np.sqrt(self.num_neurons))
        self.weights = np.zeros((num_neurons, num_neurons))
        
    #Trainig durch die Hebbsche Regel. Normiert auf die Anzahl der Neuronen. Input sind gegebene Patterns
    def train(self, patterns):
        for p in patterns:
            p = p.reshape(-1, 1)  # Ensuring pattern is a column vector
            self.weights += np.dot(p, p.T)
        np.fill_diagonal(self.weights, 0)  # No self-connections

    def predict(self, pattern, activation_function):
        s = pattern.copy().reshape(-1, 1)  # Kopie des Musters und Umformung zu Spaltenvektor
        s = np.dot(self.weights, s)
        s = activation_function(s)
        return s.flatten()  # Rückgabe als flaches Array
    
    def show_as_matrix(self,pattern):
        matrix = np.reshape(pattern, (self.matrix_size, self.matrix_size))
        return matrix
         

    def energy(self, pattern):
       
        return -0.5 * np.dot(np.dot(pattern.T, self.weights), pattern)
    
def tanh(a):
    return np.tanh(a)
    
def heviside(a):
    return np.where(a >= 0, 1, -1)


In [11]:
# Beispielmuster
patterns = [np.array([1, -1, 1, -1]), np.array([-1, 1, 1, -1])]

# Hopfield-Netzwerk mit 4 Neuronen initialisieren
network = HopfieldNetwork(num_neurons=4)

# Netzwerk mit den Mustern trainieren
network.train(patterns)

# Vorhersage für ein verrauschtes Muster
test_pattern = np.array([-1, 1, -1, -1])
result = network.predict(test_pattern, heviside)

print("Ursprüngliches Muster:", test_pattern)
print("Erkanntes Muster:", result)
print("Energie des erkannten Musters:", network.energy(result))
matrix_result = network.show_as_matrix(result)
print(matrix_result)
plt.imshow(network.show_as_matrix(patterns[0]), cmap='gray', interpolation='none')
plt.show()
plt.imshow(network.show_as_matrix(patterns[1]), cmap='gray', interpolation='none')
plt.show()
plt.imshow(network.show_as_matrix(test_pattern), cmap='gray', interpolation='none')
plt.show()
plt.imshow(matrix_result, cmap='gray', interpolation='none')
plt.show()


Ursprüngliches Muster: [-1  1 -1 -1]
Erkanntes Muster: [-1  1  1  1]
Energie des erkannten Musters: -0.0
[[-1  1]
 [ 1  1]]


In [12]:
# Beispielmuster
patterns = [
    np.array([
        [1, 1, 1, 1, 1],
        [1, -1, 1, -1, 1],
        [1, 1, 1, 1, 1],
        [-1, 1, 1, 1, -1],
        [1, -1, -1, -1, 1]
    ]),
    np.array([
        [1, 1, 1, 1, 1],
        [1, -1, 1, -1, 1],
        [1, 1, 1, 1, 1],
        [1, -1, -1, -1, 1],
        [-1, 1, 1, 1, -1]
    ])
]

# Hopfield-Netzwerk mit 4 Neuronen initialisieren
network = HopfieldNetwork(num_neurons=25)

# Netzwerk mit den Mustern trainieren
network.train(patterns)

# Vorhersage für ein verrauschtes Muster
test_pattern = np.array([
        [1, 1, 1, 1, 1],
        [1, -1, 1, -1, 1],
        [1, -1, -1, 1, 1],
        [-1, 1, -1, 1, -1],
        [-1, 1, 1, 1, -1]
    ])

result = network.predict(test_pattern,heviside)

print("Ursprüngliches Muster:", test_pattern)
print("Erkanntes Muster:", result)
print("Energie des erkannten Musters:", network.energy(result))
matrix_result = network.show_as_matrix(result)
print(matrix_result)
plt.imshow(network.show_as_matrix(patterns[0]), cmap='gray', interpolation='none')
plt.show()
plt.imshow(network.show_as_matrix(test_pattern), cmap='gray', interpolation='none')
plt.show()
plt.imshow(matrix_result, cmap='gray', interpolation='none')
plt.show()


Ursprüngliches Muster: [[ 1  1  1  1  1]
 [ 1 -1  1 -1  1]
 [ 1 -1 -1  1  1]
 [-1  1 -1  1 -1]
 [-1  1  1  1 -1]]
Erkanntes Muster: [ 1  1  1  1  1  1 -1  1 -1  1  1  1  1  1  1  1 -1 -1 -1  1 -1  1  1  1
 -1]
Energie des erkannten Musters: -300.0
[[ 1  1  1  1  1]
 [ 1 -1  1 -1  1]
 [ 1  1  1  1  1]
 [ 1 -1 -1 -1  1]
 [-1  1  1  1 -1]]


In [60]:
def capture_image():
    # Open the webcam (0 is the default device)
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return None

    # Capture a single frame
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        print("Error: Could not capture image")
        return None

    # Convert the image to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return gray

def crop_center(image, crop_size):
    """
    Schneidet einen zentralen Ausschnitt aus einem Bild.

    Parameters:
    image (numpy.ndarray): Das Eingabebild als NumPy-Array.
    crop_size (int): Die Größe des quadratischen Ausschnitts.

    Returns:
    numpy.ndarray: Der zentrale Ausschnitt des Bildes.
    """
    h, w = image.shape
    startx = w // 2 - (crop_size // 2)
    starty = h // 2 - (crop_size // 2)
    return image[starty:starty + crop_size, startx:startx + crop_size]

# Capture an image from the webcam
gray_image = capture_image()

if gray_image is not None:
    img = crop_center(gray_image,700)
    img = np.array(cv2.resize(img, (100,100)))/255
    mean = np.mean(img)

    img = np.where( img >= mean, 1, -1)
    print(img)
    plt.imshow(img)
    plt.show()
else:
    print("No image captured.")

def flip_values(matrix, probability):
    """
    Dreht die Werte in einer Matrix mit einer bestimmten Wahrscheinlichkeit um.

    Parameters:
    matrix (numpy.ndarray): Die Eingabematrix mit Werten -1 und 1.
    probability (float): Die Wahrscheinlichkeit, mit der die Werte umgedreht werden.

    Returns:
    numpy.ndarray: Die Matrix mit umgedrehten Werten basierend auf der gegebenen Wahrscheinlichkeit.
    """
    flip_mask = np.random.rand(*matrix.shape) < probability
    matrix[flip_mask] *= -1
    return matrix

noise_image = flip_values(img,0.1)
plt.imshow(img)
plt.show()



[[ 1  1  1 ... -1 -1 -1]
 [-1  1  1 ... -1 -1 -1]
 [ 1  1  1 ... -1 -1 -1]
 ...
 [ 1  1  1 ...  1  1  1]
 [ 1  1  1 ...  1  1  1]
 [ 1  1  1 ...  1  1  1]]
